# Content

1. Single Raman Spectrum Fitting
2. Single PL Spectrum Fitting
3. Raman Peaks Mapping (for 532 nm laser on WS2)
4. PL Mapping (for 532 nm laser on monolayer WS2)

# Single Raman Spectrum Fitting

In [ ]:
## N.B. This code is set to fit Raman spectra of WS2 excited by 532 nm laser
## Please modify to fit spectra of other measurements

from renishawWiRE import WDFReader
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

## Read data from .wdf file (.wdf file contains some measurement information which can be reviewed afterwards)
filename = r'path of the .wdf file'
reader = WDFReader(filename)
wavenumber = reader.xdata[810:980] ## Only focus on Raman shift = 240 to 560 1/cm, as it contains most useful peaks
                                    ## The exact range of data corresponds to the Raman shift range depends on your measurement
spectra = reader.spectra[810:980]
spectra = spectra-min(spectra) ## Subtract the background intensity for better fitting
reader.print_info()

## Read data from .txt file
# filename = r'path of the .txt file'
# data = np.loadtxt(filename) ## Depending on the structure of your data
# wavenumber = data[:,0]
# spectra = data[:,1]
# spectra = spectra-min(spectra) ## Subtract the background intensity for better fitting

## Use Cauchy-Lorentzian distribution to fit the peaks of the spectra
def Lorentzian_Raman(x, loc1, scale1, amp1, loc2, scale2, amp2, 
                        loc3, scale3, amp3, loc4, scale4, amp4, 
                        loc5, scale5, amp5, loc6, scale6, amp6,
                        loc7, scale7, amp7):
    L1 = (scale1/((x-loc1)**2+scale1**2))*amp1/np.pi ## The in-plane vibrational E12g(Γ) mode
    L2 = (scale2/((x-loc2)**2+scale2**2))*amp2/np.pi ## The out-of-plane vibrational A1g(Γ) mode
    L3 = (scale3/((x-loc3)**2+scale3**2))*amp3/np.pi ## The second-order mode of longitudinal acoustic phonon 2LA(M)
    L4 = (scale4/((x-loc4)**2+scale4**2))*amp4/np.pi ## The in-plane vibrational E12g(M) mode
    L5 = (scale5/((x-loc5)**2+scale5**2))*amp5/np.pi ## 2LA(M)-2E22g(Γ) mode
    L6 = (scale6/((x-loc6)**2+scale6**2))*amp6/np.pi ## 2LA(M)-E22g(Γ) mode
    L7 = (scale7/((x-loc7)**2+scale7**2))*amp7/np.pi ## The Si substrate
    L = L1 + L2 + L3 + L4 + L5 + L6 + L7
    return L

peak_intensity = max(spectra)
spectra_normal = spectra/peak_intensity ## Normalize the spectrum

## Set the lower and upper bounds for fitting
lower_bound = [351,0,0,
               410,0,0,
               344,0,0,
               340,0,0,
               290,0,0,
               318,0,0,
               518,0,0]
upper_bound = [360,20,10,
               425,20,10,
               355,20,10,
               351,20,10,
               300,20,10,
               330,20,10,
               525,20,10]

params, params_cov = optimize.curve_fit(Lorentzian_Raman, wavenumber, spectra_normal, 
                                        p0=[355,3,2,
                                            418,3,1,
                                            350,6,9,
                                            343,8,2,
                                            295,9,1,
                                            322,10,3,
                                            521,2,7],
                                         maxfev=6400, bounds=(lower_bound, upper_bound))

plt.figure()

plt.plot(wavenumber, spectra,'k-')

y_fit = Lorentzian_Raman(wavenumber, *params)
plt.plot(wavenumber, y_fit*peak_intensity,'b--')

for i in range(round(len(lower_bound)/3)):
    y_fit_single = (params[3*i+1]/((wavenumber-params[3*i])**2+params[3*i+1]**2))*params[3*i+2]/np.pi
    plt.plot(wavenumber, y_fit_single*peak_intensity,'r--')
    print('loc =',params[3*i],', scale =',params[3*i+1],', amp =',params[3*i+2])
    
plt.xlabel('Raman Shift (1/cm)')
plt.ylabel('Counts (a.u.)')
plt.xlim(240,560)
# plt.xticks(fountsize=20)
plt.yticks([])
plt.show()

# Single PL Spectrum Fitting

In [ ]:
## N.B. This code is set to fit PL spectra of monolayer WS2 excited by 532 nm laser
## Please modify to fit spectra of other measurements

from renishawWiRE import WDFReader
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

## Read data from .wdf file (.wdf file contains some measurement information which can be reviewed afterwards)
filename = r'path of the .wdf file'
reader = WDFReader(filename)
energy = reader.xdata[:]
intensity = reader.spectra[:]
reader.print_info()

## Read data from .txt file
# filename = r'path of the .txt file'
# data = np.loadtxt(filename) ## Depending on the structure of your data
# energy = data[:,0]
# intensity = data[:,1]

## Use Cauchy-Lorentzian distribution to fit the peaks of the spectra
def Lorentzian_PL(x, loc1, scale1, amp1, loc2, scale2, amp2):
    L1 = (scale1/((x-loc1)**2+scale1**2))*amp1/np.pi ## Exciton
    L2 = (scale2/((x-loc2)**2+scale2**2))*amp2/np.pi ## Trion
    L = L1 + L2
    return L

peak_intensity = max(intensity)
intensity_normal = intensity/peak_intensity ## Normalize the spectrum

## Set the lower and upper bounds for fitting
lower_bound = [1.95,0,0,
               1.90,0,0]
upper_bound = [2.1,0.05,10,
               2,0.1,10]

params, params_cov = optimize.curve_fit(Lorentzian_PL, energy, intensity_normal, 
                                         p0=[2,0.02,0.03,
                                             1.98,0.03,0.04],
                                         maxfev=6400, bounds=(lower_bound,upper_bound))

print('loc_Exciton =',params[0],', scale_Exciton =',params[1],', amp_Exciton =',params[2])
print('loc_Trion =',params[3],', scale_Trion =',params[4],', amp_Trion =',params[5])

y_fit = Lorentzian_PL(energy, *params)
y_fit_1 = (params[1]/((energy-params[0])**2+params[1]**2))*params[2]/np.pi
y_fit_2 = (params[4]/((energy-params[3])**2+params[4]**2))*params[5]/np.pi

plt.figure()
plt.plot(energy, intensity,'k-')
plt.plot(energy,y_fit_1*peak_intensity,'r--')
plt.plot(energy,y_fit_2*peak_intensity,'r--')
plt.plot(energy,y_fit*peak_intensity,'b--')
plt.xlabel('Energy (eV)')
plt.ylabel('Counts (a.u.)')
plt.xticks([1.90,1.95,2.00,2.05])
plt.yticks([])
# plt.xlim([1.89,2.05])
plt.show()

# Raman Peaks Mapping (for 532 nm laser on WS2)

In [ ]:
## N.B. This code is set to map Raman peaks distance and ratio of WS2 (A1g, E2g) excited by 532 nm laser
## Please modify to fit spectra of other measurements

from renishawWiRE import WDFReader
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

## Read data from .wdf file (.wdf file contains some measurement information which can be reviewed afterwards)
filename = r'path of the .wdf mapping file'
reader = WDFReader(filename)
print(reader.map_info)
## Show optical image of the mapping area (marked by the rectangle)
img = Image.open(reader.img)
cb = reader.img_cropbox
fig, ax = plt.subplots(1) 
ax.imshow(img) 
rect = patches.Rectangle((cb[0],cb[1]),cb[2]-cb[0],cb[3]-cb[1],linewidth=1,edgecolor='r',facecolor='none') 
ax.add_patch(rect) 
plt.show()

X = reader.map_shape[0] ## How many steps in X
Y = reader.map_shape[1] ## How many steps in Y

## Read data from .txt file
# filename = r'path of the .txt mapping file'
# mapping = np.loadtxt(filename)
# spectrum_length = np.where(mapping[:,0] == np.unique(mapping[:,0])[1])[0][0] ## How many data points per spectrum
#                                                                              ## Depending on the structure of your data
# X = len(np.unique(mapping[:,0])) ## How many steps in X
# Y = len(np.unique(mapping[:,1])) ## How many steps in Y

## Create .txt files to save fitting results
# E2g_Position = r'path\E2g_Position.txt'
# E2g_Peak = r'path\E2g_Peak.txt'
# A1g_Position = r'path\A1g_Position.txt'
# A1g_Peak = r'path\A1g_Peak.txt'

## Create lists to store fitting results
Peaks_distance = []
Peaks_ratio = []

## Use Cauchy-Lorentzian distribution to fit the peaks of the spectra
## For each component of the function, please refer to Part 1 Single Raman Spectrum Fitting
def Lorentzian_Raman(x, loc1, scale1, amp1, loc2, scale2, amp2, 
                        loc3, scale3, amp3, loc4, scale4, amp4, 
                        loc5, scale5, amp5, loc6, scale6, amp6, 
                        loc7, scale7, amp7):
    L1 = (scale1/((x-loc1)**2+scale1**2))*amp1/np.pi
    L2 = (scale2/((x-loc2)**2+scale2**2))*amp2/np.pi
    L3 = (scale3/((x-loc3)**2+scale3**2))*amp3/np.pi
    L4 = (scale4/((x-loc4)**2+scale4**2))*amp4/np.pi
    L5 = (scale5/((x-loc5)**2+scale5**2))*amp5/np.pi
    L6 = (scale6/((x-loc6)**2+scale6**2))*amp6/np.pi
    L7 = (scale7/((x-loc7)**2+scale7**2))*amp7/np.pi
    L = L1 + L2 + L3 + L4 + L5 + L6 + L7
    return L

## Set the lower and upper bounds for fitting
lower_bound = [351,0,0,
               410,0,0,
               344,0,0,
               340,0,0,
               290,0,0,
               318,0,0,
               518,0,0]
upper_bound = [360,20,10,
               425,20,10,
               355,20,10,
               351,20,10,
               300,20,10,
               330,20,10,
               525,20,10]

for j in range(Y):
    for i in range(X):
        try:
            ## When read data from .wdf file
            wavenumber = reader.xdata[800:880] ## Only focus on the E2g and A1g peaks
                                               ## The exact range of data depends on your measurement
            spectra = reader.spectra[j][i][800:880]

            ## When read data from .txt file
#             wavenumber = mapping[(j*i)*spectrum_length:(j*i+1)*spectrum_length,2][800:880] ## Only focus on the E2g and A1g peaks
#                                                                                            ## The exact range depends on your measurement
#             spectra = mapping[(j*i)*spectrum_length:(j*i+1)*spectrum_length,3][800:880]
            
            spectra_max = max(spectra)
            spectra_min = min(spectra)
            
            spectra = spectra - spectra_min ## Subtract the background intensity for better fitting
            spectra_normal = spectra/spectra_max ## Normalise the spectrum

            params, params_cov = optimize.curve_fit(Lorentzian_Raman, wavenumber, spectra_normal, 
                                                    p0=[355,3,2,
                                                        418,3,1,
                                                        350,6,9,
                                                        343,8,2,
                                                        295,9,1,
                                                        322,10,3,
                                                        521,2,7],
                                                     maxfev=6400, bounds=(lower_bound,upper_bound))
            yfit_E2g = (params[1]/((wavenumber-params[0])**2+params[1]**2))*params[2]/np.pi
            yfit_A1g = (params[4]/((wavenumber-params[3])**2+params[4]**2))*params[5]/np.pi
            peak_E2g = max(yfit_E2g)*spectra_max
            peak_A1g = max(yfit_A1g)*spectra_max
            
            ## Plot the fitting result if want to
#             plt.figure()
#             plt.plot(wavenumber, spectra,'k-')
#             plt.plot(wavenumber,yfit_E2g*spectra_max,'r--') ## Please don't be confused with 2LA(M) (not plotted)
#             plt.plot(wavenumber,yfit_A1g*spectra_max,'b--')
#             plt.show()
            
        except RuntimeError:
            params[0] = 0
            params[6] = 0
            peak_E2g = 0
            peak_A1g = 1

        Peaks_distance.append(params[3]-params[0])
        Peaks_ratio.append(peak_E2g/peak_A1g)

        ## Save the fitting results to corresponding .txt files
#         data1=open(E2g_Position,'a+')
#         print(params[0], file=data1)
#         data1.close()

#         data2=open(E2g_Peak,'a+')
#         print(peak_E2g, file=data2)
#         data2.close()

#         data3=open(A1g_Position,'a+')
#         print(params[6], file=data3)
#         data3.close()

#         data4=open(A1g_Peak,'a+')
#         print(peak_A1g, file=data4)
#         data4.close()

plt.figure()
Peaks_distance = np.reshape(Peaks_distance,(Y,X)) 
im = plt.imshow(Peaks_distance, cmap='Spectral', aspect='auto', vmin=60, vmax=67)
cbar = plt.colorbar(im, label='A1g-E2g (1/cm)', ticks=[62, 63, 64, 65, 66, 67, 68])
# cbar.ax.tick_params(labelsize=15)  ## Set the font size of colorbar labels
plt.axis('off')
plt.show()

plt.figure()
Peaks_ratio = np.reshape(Peaks_ratio,(Y,X)) 
img = plt.imshow(Peaks_ratio, cmap='Spectral', aspect='auto', vmin=0, vmax=1.2)
cbar = plt.colorbar(img, label='E2g/A1g', ticks=[0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
# cbar.ax.tick_params(labelsize=15)  ## Set the font size of colorbar labels
plt.axis('off')
plt.show()

# PL Mapping (for 532 nm laser on monolayer WS2)

In [ ]:
## N.B. This code is set to map the PL peaks of monolayer WS2 (Exciton, Trion) excited by 532 nm laser
## Please modify to fit spectra of other measurements

from renishawWiRE import WDFReader
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

## Read data from .wdf file (.wdf file contains some measurement information which can be reviewed afterwards)
filename = r'path of the .wdf mapping file'
reader = WDFReader(filename)
print(reader.map_info)
## Show optical image of the mapping area (marked by the rectangle)
img = Image.open(reader.img)
cb = reader.img_cropbox
fig, ax = plt.subplots(1) 
ax.imshow(img) 
rect = patches.Rectangle((cb[0],cb[1]),cb[2]-cb[0],cb[3]-cb[1],linewidth=1,edgecolor='r',facecolor='none') 
ax.add_patch(rect) 
plt.show()

X = reader.map_shape[0] ## How many steps in X
Y = reader.map_shape[1] ## How many steps in Y

## Read data from .txt file
# filename = r'path of the .txt mapping file'
# mapping = np.loadtxt(filename)
# spectrum_length = np.where(mapping[:,0] == np.unique(mapping[:,0])[1])[0][0] ## How many data points per spectrum
#                                                                              ## Depending on the structure of your data
# X = len(np.unique(mapping[:,0])) ## How many steps in X
# Y = len(np.unique(mapping[:,1])) ## How many steps in Y

## Create .txt files to save fitting results
# Fitted_Params = r'path\Fitted_Params.txt'
# add_headers=open(Fitted_Params,'a+')
# print('loc_Exciton\t','scale_Exciton\t','amp_Exciton\t','loc_Trion\t','scale_Trion\t','amp_Trion\t', file=add_headers)
# add_headers.close()

## Create lists to store fitting results
Exciton_position = []
Peak_intensity = [] ## Peak intensity of the fitted measured curve (i.e. Exciton + Trion combined)

## Use Cauchy-Lorentzian distribution to fit the peaks of the spectra
def Lorentzian_PL(x, loc1, scale1, amp1, loc2, scale2, amp2):
    L1 = (scale1/((x-loc1)**2+scale1**2))*amp1/np.pi ## Exciton
    L2 = (scale2/((x-loc2)**2+scale2**2))*amp2/np.pi ## Trion
    L = L1 + L2
    return L

## Set the lower and upper bounds for fitting
lower_bound = [1.95,0,0,
               1.90,0,0]
upper_bound = [2.1,0.05,10,
               2,0.1,10]

for j in range(Y):
    for i in range(X):
        try:
            ## When read data from .wdf file
            energy = reader.xdata[:]
            intensity = reader.spectra[j][i][:]

            ## When read data from .txt file
#             energy = mapping[(j*i)*spectrum_length:(j*i+1)*spectrum_length,2][:]
#             intensity = mapping[(j*i)*spectrum_length:(j*i+1)*spectrum_length,3][:]
            
            intensity_max = max(intensity)
            intensity_min = min(intensity)
            
            intensity = intensity - intensity_min ## Subtract the background intensity for better fitting
            intensity_normal = intensity/intensity_max ## Normalise the spectrum

            params, params_cov = optimize.curve_fit(Lorentzian_PL, energy, intensity_normal, 
                                                    p0=[2,0.02,0.03,
                                                        1.98,0.03,0.04],
                                                     maxfev=6400, bounds=(lower_bound,upper_bound))
            yfit = Lorentzian_PL(energy, *params)
            peak = max(yfit)*intensity_max
            
            ## Plot the fitting result if want to
#             plt.figure()
#             plt.plot(energy, intensity,'k-')
#             yfit_1 = (params[1]/((energy-params[0])**2+params[1]**2))*params[2]/np.pi
#             yfit_2 = (params[4]/((energy-params[3])**2+params[4]**2))*params[5]/np.pi
#             plt.plot(energy,yfit_1*intensity_max,'r--')
#             plt.plot(energy,yfit_2*intensity_max,'r--')
#             plt.plot(energy,yfit*intensity_max,'b--')
#             plt.show()
            
        except RuntimeError:
            params[0] = 0
            peak = 0

        Exciton_position.append(params[0])
        Peak_intensity.append(peak)

        ## Save the fitting results to the .txt file
#         data=open(Fitted_Params,'a+')
#         print(*params, file=data)
#         data.close()

plt.figure()
Exciton_position = np.reshape(Exciton_position,(Y,X)) 
im = plt.imshow(Exciton_position, cmap='Spectral', aspect='auto', vmin=1.95, vmax=2.01)
cbar = plt.colorbar(im, label='Exciton Position', ticks=[1.96,1.97,1.98,1.99,2.00])
# cbar.ax.tick_params(labelsize=15)  ## Set the font size of colorbar labels
plt.axis('off')
plt.show()

plt.figure()
Peak_intensity = np.reshape(Peak_intensity,(Y,X)) 
img = plt.imshow(Peak_intensity, cmap='Spectral', aspect='auto')
cbar = plt.colorbar(img, label='Intensity', ticks=[])
# cbar.ax.tick_params(labelsize=15)  ## Set the font size of colorbar labels
plt.axis('off')
plt.show()